<font size=8> Network Tests


In [1]:
#Common Python Packages
from pickle import TRUE
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import timeout_decorator

#dynapse2 spikegen imports
from lib.dynapse2_util import *
from lib.dynapse2_network import Network
from lib.dynapse2_spikegen import send_events,get_fpga_time, send_virtual_events, poisson_gen, isi_gen,regular_gen
from lib.dynapse2_raster import *
from lib.dynapse2_obj import *

#Statistics imports
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy.signal import butter, filtfilt
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from itertools import product

#my imports
from adaptation import pc_single,ff,pc_pv_sst,ff_network
from adaptation_lib.spike_stats import *
from adaptation_lib.dynapse_setup import *
from adaptation_lib.graphing import *
from configs import neuron_configs_bio

# Determine the user's home directory
home_directory = os.path.expanduser("~")
# Path to the Documents directory
documents_path = os.path.join(home_directory, "Documents")
# Path to the dynapse-se2-data directory within Documents
save_directory_path = os.path.join(documents_path, "dynapse-se2-data")
# Path to the data
data_type_path = os.path.join(save_directory_path, "EI_tests")

@timeout_decorator.timeout(40)# Set a 5-second timeout
def run_simulation(neuron_config):
    return pc_pv_sst.pc_pv_sst(
        board=board,
        profile_path=profile_path,
        number_of_chips=number_of_chips,
        neuron_config=neuron_config
    )

In [2]:
[board,profile_path,number_of_chips]=obtain_board()

<font size=8> SWEEP

In [3]:

neuron_config=neuron_configs_bio.config()
neuron_config['Input_SST']=0#.1#.1
neuron_config['PC_SST']=0#.1

neuron_config['PC_Adaptation']=True
neuron_config['duration']=1
neuron_config['input_type']='Regular'
neuron_config['in_freq']=20

#input frequencies and conneciton ratiors
input_frequencies=np.arange(1,31,1)
connection_ratios=np.arange(0,.7,.1)

data=os.path.join(data_type_path, "EE_sweep_regular.npy")
# Check if the data file exists
if os.path.exists(data):
    # Load existing data
    simulation_data = np.load(data, allow_pickle=True).item()
else:
    if not os.path.exists(data):
        # If not, initialize with NaNs for every combination
        simulation_data = {
            "cv_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "cv_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "mean_pc_rates": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "input_frequencies": np.arange(1, 21, 1),
            "connection_ratios": np.arange(0, .7, .1)
        }

def contains_nan(input_freq, connection_ratio):
    input_idx = np.argmin(np.abs(np.array(simulation_data["input_frequencies"]) - input_freq))
    conn_idx = np.argmin(np.abs(np.array(simulation_data["connection_ratios"]) - connection_ratio))
    
    index = len(simulation_data["connection_ratios"]) * input_idx + conn_idx
    
    return np.isnan(simulation_data["cv_values_pc"][index]) or \
           np.isnan(simulation_data["cv_values_pv"][index]) or \
           np.isnan(simulation_data["synchrony_values_pc"][index]) or \
           np.isnan(simulation_data["synchrony_values_pv"][index]) or \
           np.isnan(simulation_data["mean_pc_rates"][index])


for input_freq in simulation_data["input_frequencies"]:
    neuron_config['input_frequency'] = input_freq
    for connection_ratio in simulation_data["connection_ratios"]:
        # Only run the simulation if the data for this configuration contains NaN values
        if contains_nan(input_freq, connection_ratio):
            neuron_config['PC_PC'] = connection_ratio
            try:
                [output_events, test_config] = run_simulation(neuron_config)

                cv_temp, synchrony_temp = run_dynamic_anal(output_events, test_config)

                rates = spike_count(output_events=output_events)
                PC_rates, _, _ = pop_rates(rates, test_config)

                # Update the results in the loaded/initialized data
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                simulation_data["cv_values_pc"][index] = cv_temp[0]
                simulation_data["cv_values_pv"][index] = cv_temp[1]
                simulation_data["synchrony_values_pc"][index] = synchrony_temp[0]
                simulation_data["synchrony_values_pv"][index] = synchrony_temp[1]
                simulation_data["mean_pc_rates"][index] = np.mean(PC_rates)
            except timeout_decorator.TimeoutError:
                print("Simulation timed out for input frequency %d and connection ratio %.2f" % (input_freq, connection_ratio))
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                # You might want to update the corresponding values to NaN if timeout occurs. Depends on your preference.
                simulation_data["cv_values_pc"][index] = np.nan
                simulation_data["cv_values_pv"][index] = np.nan
                simulation_data["synchrony_values_pc"][index] = np.nan
                simulation_data["synchrony_values_pv"][index] = np.nan
                simulation_data["mean_pc_rates"][index] = np.nan

# Save the updated data
if not os.path.exists(data_type_path):
    os.makedirs(data_type_path)
np.save(os.path.join(data_type_path, "EE_sweep_regular.npy"), simulation_data)



Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 1 and connection ratio 0.60
Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 2 and connection ratio 0.60
Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 3 and connection ratio 0.60
Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 4 and connection ratio 0.60
Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 5 and connection ratio 0.60
Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 6 and connection ratio 0.60
Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 7 and connection ratio 0.60
Setting Latches
Setting up network
Adding connections
Simulation timed out for input frequency 8 and connection ratio 0.60
Setting Latches


In [ ]:
import numpy as np
import os
neuron_config=neuron_configs_bio.config()


neuron_config['Input_SST']=0#.1#.1

neuron_config['PC_SST']=0#.1

neuron_config['PC_Adaptation']=True
neuron_config['duration']=1
neuron_config['input_type']='Regular'
neuron_config['in_freq']=20

#input frequencies and conneciton ratiors
input_frequencies=np.arange(1,31,1)
connection_ratios=np.arange(0,.7,.1)

data=os.path.join(data_type_path, "EI_sweep_regular.npy")
# Check if the data file exists
if os.path.exists(data):
    # Load existing data
    simulation_data = np.load(data, allow_pickle=True).item()
else:
    if not os.path.exists(data):
        # If not, initialize with NaNs for every combination
        simulation_data = {
            "cv_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "cv_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "mean_pc_rates": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "input_frequencies": np.arange(1, 21, 1),
            "connection_ratios": np.arange(0, .7, .1)
        }

def contains_nan(input_freq, connection_ratio):
    input_idx = np.argmin(np.abs(np.array(simulation_data["input_frequencies"]) - input_freq))
    conn_idx = np.argmin(np.abs(np.array(simulation_data["connection_ratios"]) - connection_ratio))
    
    index = len(simulation_data["connection_ratios"]) * input_idx + conn_idx
    
    return np.isnan(simulation_data["cv_values_pc"][index]) or \
           np.isnan(simulation_data["cv_values_pv"][index]) or \
           np.isnan(simulation_data["synchrony_values_pc"][index]) or \
           np.isnan(simulation_data["synchrony_values_pv"][index]) or \
           np.isnan(simulation_data["mean_pc_rates"][index])


for input_freq in simulation_data["input_frequencies"]:
    neuron_config['input_frequency'] = input_freq
    for connection_ratio in simulation_data["connection_ratios"]:
        # Only run the simulation if the data for this configuration contains NaN values
        if contains_nan(input_freq, connection_ratio):
            neuron_config['PV_PC'] = connection_ratio
            try:
                [output_events, test_config] = run_simulation(neuron_config)

                cv_temp, synchrony_temp = run_dynamic_anal(output_events, test_config)

                rates = spike_count(output_events=output_events)
                PC_rates, _, _ = pop_rates(rates, test_config)

                # Update the results in the loaded/initialized data
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                simulation_data["cv_values_pc"][index] = cv_temp[0]
                simulation_data["cv_values_pv"][index] = cv_temp[1]
                simulation_data["synchrony_values_pc"][index] = synchrony_temp[0]
                simulation_data["synchrony_values_pv"][index] = synchrony_temp[1]
                simulation_data["mean_pc_rates"][index] = np.mean(PC_rates)
            except timeout_decorator.TimeoutError:
                print("Simulation timed out for input frequency %d and connection ratio %.2f" % (input_freq, connection_ratio))
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                # You might want to update the corresponding values to NaN if timeout occurs. Depends on your preference.
                simulation_data["cv_values_pc"][index] = np.nan
                simulation_data["cv_values_pv"][index] = np.nan
                simulation_data["synchrony_values_pc"][index] = np.nan
                simulation_data["synchrony_values_pv"][index] = np.nan
                simulation_data["mean_pc_rates"][index] = np.nan

# Save the updated data
if not os.path.exists(data_type_path):
    os.makedirs(data_type_path)
np.save(os.path.join(data_type_path, "EI_sweep_regular.npy"), simulation_data)



Setting Latches
Setting up network
Adding connections

Setting monitors
20 100 10

All configurations done!

Input events created
initilize run dynapse
dummy events sent

getting fpga time


setting virtual neurons

Simulation done
PC_CV_average: 0.02 PV_CV_average: 0.16 SST_CV_average: 0.0
PC_synchrony: 0.1 PV_CV_average: 0.24 SST_CV_average: nan
Chip 0 core 0:
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 0],
[49, 52, 45, 53, 51, 52, 58, 45, 51, 49, 50, 50, 43, 47, 53, 49],
[51, 52, 48, 47, 52, 52, 55, 52, 48, 50, 52, 53, 49, 47, 47, 56],
[45, 54, 52, 56, 50, 56, 54, 54, 50, 55, 57, 55, 39, 52, 46, 54],
[49, 47, 51, 49, 52, 38, 48, 51, 52, 54, 55, 50, 53, 43, 60, 52],
[38, 47, 46, 54, 47, 44, 44, 42, 56, 49, 51, 43, 49, 52, 51, 54],
[47, 52, 57, 49, 40, 46, 45, 51, 57, 52, 50, 43, 55, 39, 53, 44],
[51, 53, 49, 47, 0, 56, 85, 86, 93, 79, 102, 32, 79, 109, 97, 125],
[16, 142, 113, 97, 77, 122, 22, 55, 101, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import numpy as np
import os
neuron_config=neuron_configs_bio.config()


neuron_config['Input_SST']=0#.1#.1

neuron_config['PC_SST']=0#.1

neuron_config['PC_Adaptation']=True
neuron_config['duration']=1
neuron_config['input_type']='Regular'
neuron_config['in_freq']=20

#input frequencies and conneciton ratiors
input_frequencies=np.arange(1,31,1)
connection_ratios=np.arange(0,.7,.1)

data=os.path.join(data_type_path, "EI_sweep_poisson.npy")
# Check if the data file exists
if os.path.exists(data):
    # Load existing data
    simulation_data = np.load(data, allow_pickle=True).item()
else:
    if not os.path.exists(data):
        # If not, initialize with NaNs for every combination
        simulation_data = {
            "cv_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "cv_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "mean_pc_rates": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "input_frequencies": np.arange(1, 21, 1),
            "connection_ratios": np.arange(0, .7, .1)
        }

def contains_nan(input_freq, connection_ratio):
    input_idx = np.argmin(np.abs(np.array(simulation_data["input_frequencies"]) - input_freq))
    conn_idx = np.argmin(np.abs(np.array(simulation_data["connection_ratios"]) - connection_ratio))
    
    index = len(simulation_data["connection_ratios"]) * input_idx + conn_idx
    
    return np.isnan(simulation_data["cv_values_pc"][index]) or \
           np.isnan(simulation_data["cv_values_pv"][index]) or \
           np.isnan(simulation_data["synchrony_values_pc"][index]) or \
           np.isnan(simulation_data["synchrony_values_pv"][index]) or \
           np.isnan(simulation_data["mean_pc_rates"][index])


for input_freq in simulation_data["input_frequencies"]:
    neuron_config['input_frequency'] = input_freq
    for connection_ratio in simulation_data["connection_ratios"]:
        # Only run the simulation if the data for this configuration contains NaN values
        if contains_nan(input_freq, connection_ratio):
            neuron_config['PV_PC'] = connection_ratio
            try:
                [output_events, test_config] = run_simulation(neuron_config)

                cv_temp, synchrony_temp = run_dynamic_anal(output_events, test_config)

                rates = spike_count(output_events=output_events)
                PC_rates, _, _ = pop_rates(rates, test_config)

                # Update the results in the loaded/initialized data
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                simulation_data["cv_values_pc"][index] = cv_temp[0]
                simulation_data["cv_values_pv"][index] = cv_temp[1]
                simulation_data["synchrony_values_pc"][index] = synchrony_temp[0]
                simulation_data["synchrony_values_pv"][index] = synchrony_temp[1]
                simulation_data["mean_pc_rates"][index] = np.mean(PC_rates)
            except timeout_decorator.TimeoutError:
                print("Simulation timed out for input frequency %d and connection ratio %.2f" % (input_freq, connection_ratio))
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                # You might want to update the corresponding values to NaN if timeout occurs. Depends on your preference.
                simulation_data["cv_values_pc"][index] = np.nan
                simulation_data["cv_values_pv"][index] = np.nan
                simulation_data["synchrony_values_pc"][index] = np.nan
                simulation_data["synchrony_values_pv"][index] = np.nan
                simulation_data["mean_pc_rates"][index] = np.nan

# Save the updated data
if not os.path.exists(data_type_path):
    os.makedirs(data_type_path)
np.save(os.path.join(data_type_path, "EI_sweep_poisson.npy"), simulation_data)



Setting Latches
Setting up network
Adding connections

Setting monitors
20 100 10

All configurations done!

Input events created
initilize run dynapse
dummy events sent

getting fpga time


setting virtual neurons

Simulation done
PC_CV_average: 0.02 PV_CV_average: 0.14 SST_CV_average: 0.0
PC_synchrony: 0.1 PV_CV_average: 0.25 SST_CV_average: nan
Chip 0 core 0:
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 0],
[49, 52, 44, 53, 50, 52, 59, 45, 51, 50, 49, 48, 42, 47, 53, 49],
[50, 53, 47, 48, 51, 51, 54, 53, 47, 50, 51, 53, 49, 48, 46, 56],
[45, 53, 51, 56, 50, 56, 55, 52, 50, 55, 57, 55, 39, 52, 46, 54],
[48, 46, 50, 50, 52, 38, 48, 51, 52, 53, 55, 50, 52, 41, 60, 52],
[38, 45, 47, 53, 47, 43, 43, 42, 56, 49, 51, 44, 49, 51, 50, 52],
[46, 52, 56, 48, 39, 45, 44, 51, 57, 51, 49, 42, 54, 41, 53, 43],
[49, 52, 49, 47, 0, 38, 35, 106, 90, 75, 126, 21, 107, 119, 53, 142],
[21, 133, 126, 98, 67, 130, 24, 24, 131, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:

neuron_config=neuron_configs_bio.config()
neuron_config['Input_SST']=0#.1#.1
neuron_config['PC_SST']=0#.1

neuron_config['PC_Adaptation']=True
neuron_config['duration']=1
neuron_config['input_type']='Poisson'
neuron_config['in_freq']=20

#input frequencies and conneciton ratiors
input_frequencies=np.arange(1,31,1)
connection_ratios=np.arange(0,.7,.1)

data=os.path.join(data_type_path, "EE_sweep_poisson.npy")
# Check if the data file exists
if os.path.exists(data):
    # Load existing data
    simulation_data = np.load(data, allow_pickle=True).item()
else:
    if not os.path.exists(data):
        # If not, initialize with NaNs for every combination
        simulation_data = {
            "cv_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "cv_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pc": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "synchrony_values_pv": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "mean_pc_rates": [np.nan] * len(input_frequencies) * len(connection_ratios),
            "input_frequencies": np.arange(1, 21, 1),
            "connection_ratios": np.arange(0, .7, .1)
        }

def contains_nan(input_freq, connection_ratio):
    input_idx = np.argmin(np.abs(np.array(simulation_data["input_frequencies"]) - input_freq))
    conn_idx = np.argmin(np.abs(np.array(simulation_data["connection_ratios"]) - connection_ratio))
    
    index = len(simulation_data["connection_ratios"]) * input_idx + conn_idx
    
    return np.isnan(simulation_data["cv_values_pc"][index]) or \
           np.isnan(simulation_data["cv_values_pv"][index]) or \
           np.isnan(simulation_data["synchrony_values_pc"][index]) or \
           np.isnan(simulation_data["synchrony_values_pv"][index]) or \
           np.isnan(simulation_data["mean_pc_rates"][index])


for input_freq in simulation_data["input_frequencies"]:
    neuron_config['input_frequency'] = input_freq
    for connection_ratio in simulation_data["connection_ratios"]:
        # Only run the simulation if the data for this configuration contains NaN values
        if contains_nan(input_freq, connection_ratio):
            neuron_config['PC_PC'] = connection_ratio
            try:
                [output_events, test_config] = run_simulation(neuron_config)

                cv_temp, synchrony_temp = run_dynamic_anal(output_events, test_config)

                rates = spike_count(output_events=output_events)
                PC_rates, _, _ = pop_rates(rates, test_config)

                # Update the results in the loaded/initialized data
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                simulation_data["cv_values_pc"][index] = cv_temp[0]
                simulation_data["cv_values_pv"][index] = cv_temp[1]
                simulation_data["synchrony_values_pc"][index] = synchrony_temp[0]
                simulation_data["synchrony_values_pv"][index] = synchrony_temp[1]
                simulation_data["mean_pc_rates"][index] = np.mean(PC_rates)
            except timeout_decorator.TimeoutError:
                print("Simulation timed out for input frequency %d and connection ratio %.2f" % (input_freq, connection_ratio))
                index = len(simulation_data["connection_ratios"]) * list(simulation_data["input_frequencies"]).index(input_freq) + list(simulation_data["connection_ratios"]).index(connection_ratio)
                # You might want to update the corresponding values to NaN if timeout occurs. Depends on your preference.
                simulation_data["cv_values_pc"][index] = np.nan
                simulation_data["cv_values_pv"][index] = np.nan
                simulation_data["synchrony_values_pc"][index] = np.nan
                simulation_data["synchrony_values_pv"][index] = np.nan
                simulation_data["mean_pc_rates"][index] = np.nan

# Save the updated data
if not os.path.exists(data_type_path):
    os.makedirs(data_type_path)
np.save(os.path.join(data_type_path, "EE_sweep_poisson.npy"), simulation_data)

<font size=8> plotting

In [ ]:


# Load the data
data_path = os.path.join(data_type_path, "simulation_data_1.npy")
data = np.load(data_path, allow_pickle=True).item()

# Extract the data
cv_values_pc = np.array(data['cv_values_pc']).reshape(len(data['input_frequencies']), -1)[::-1]
synchrony_values_pc = np.array(data['synchrony_values_pc']).reshape(len(data['input_frequencies']), -1)[::-1]
mean_pc_rates = np.array(data['mean_pc_rates']).reshape(len(data['input_frequencies']), -1)[::-1]

input_frequencies = data['input_frequencies'][::-1]
connection_ratios = [round(x, 1) for x in data['connection_ratios']]
y_ticks = np.arange(0, len(input_frequencies), 5)
y_ticklabels = [input_frequencies[i] for i in y_ticks]

# Plotting
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# CV Values PC
sns.heatmap(cv_values_pc, ax=ax[0], cmap='viridis', cbar_kws={'label': 'CV Values PC'})
ax[0].set_xticks(np.arange(len(connection_ratios)) + 0.5)
ax[0].set_xticklabels(connection_ratios)
ax[0].set_yticks(y_ticks)
ax[0].set_yticklabels(y_ticklabels)
ax[0].set_xlabel('Connection Ratios')
ax[0].set_ylabel('Input Frequencies')
ax[0].set_title('CV Values PC')

# Synchrony Values PC
sns.heatmap(synchrony_values_pc, ax=ax[1], cmap='viridis', cbar_kws={'label': 'Synchrony Values PC'})
ax[1].set_xticks(np.arange(len(connection_ratios)) + 0.5)
ax[1].set_xticklabels(connection_ratios)
ax[1].set_yticks(y_ticks)
ax[1].set_yticklabels(y_ticklabels)
ax[1].set_xlabel('Connection Ratios')
ax[1].set_ylabel('Input Frequencies')
ax[1].set_title('Synchrony Values PC')

# Mean Firing Rates PC
sns.heatmap(mean_pc_rates, ax=ax[2], cmap='viridis', cbar_kws={'label': 'Mean Firing Rates PC'})
ax[2].set_xticks(np.arange(len(connection_ratios)) + 0.5)
ax[2].set_xticklabels(connection_ratios)
ax[2].set_yticks(y_ticks)
ax[2].set_yticklabels(y_ticklabels)
ax[2].set_xlabel('Connection Ratios')
ax[2].set_ylabel('Input Frequencies')
ax[2].set_title('Mean Firing Rates PC')

plt.tight_layout()
plt.show()


KeyError: 'cv_values_pc'

In [ ]:


FONT_SIZE = 12
plt.rcParams['font.size'] = FONT_SIZE

def plot_heatmap(data_matrix, ax, cmap, xlabel, ylabel, title, cbar_label,
                 x_ticks, x_ticklabels, y_ticks, y_ticklabels):
    """
    Helper function to plot a heatmap.
    """
    sns.heatmap(data_matrix, ax=ax, cmap=cmap, cbar_kws={'label': cbar_label})
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_ticklabels)
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_ticklabels)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)

def visualize_simulation_data(data_type_path):
    """
    Load and visualize simulation data.
    """
    # Load the data
    data_path = os.path.join(data_type_path, "simulation_data_1.npy")
    data = np.load(data_path, allow_pickle=True).item()

    # Extract the data
    cv_values_pc = np.array(data['cv_values_pc']).reshape(len(data['input_frequencies']), -1)[::-1]
    synchrony_values_pc = np.array(data['synchrony_values_pc']).reshape(len(data['input_frequencies']), -1)[::-1]
    mean_pc_rates = np.array(data['mean_pc_rates']).reshape(len(data['input_frequencies']), -1)[::-1]

    input_frequencies = data['input_frequencies'][::-1]
    connection_ratios = [round(x, 1) for x in data['connection_ratios']]
    
    y_ticks = np.arange(0, len(input_frequencies), 5)
    y_ticklabels = [input_frequencies[i] for i in y_ticks]
    x_ticks = np.arange(len(connection_ratios)) + 0.5
    x_ticklabels = connection_ratios
    
    # Plotting
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    
    plot_heatmap(cv_values_pc, ax[0], 'viridis', 'Connection Ratios', 'Input Frequencies', 'CV Values PC', 'CV Values PC', x_ticks, x_ticklabels, y_ticks, y_ticklabels)
    plot_heatmap(synchrony_values_pc, ax[1], 'viridis', 'Connection Ratios', 'Input Frequencies', 'Synchrony Values PC', 'Synchrony Values PC', x_ticks, x_ticklabels, y_ticks, y_ticklabels)
    plot_heatmap(mean_pc_rates, ax[2], 'viridis', 'Connection Ratios', 'Input Frequencies', 'Mean Firing Rates PC', 'Mean Firing Rates PC', x_ticks, x_ticklabels, y_ticks, y_ticklabels)
    
    plt.tight_layout()
    plt.show()

# Example usage
# visualize_simulation_data("/path/to/data/type")
